### Import Packages

In [2]:
import os
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import seaborn as sns
from nltk.corpus import stopwords
import string
from nltk import word_tokenize,ne_chunk,pos_tag,wordpunct_tokenize
import rake_nltk
from nltk.util import ngrams
from nltk.stem import WordNetLemmatizer,PorterStemmer
from itertools import chain, groupby, product
from collections import defaultdict, Counter
import networkx as nx
from networkx.algorithms import community
from gensim.models import Word2Vec,KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models.ldamodel import LdaModel
import gensim
import statsmodels.api as sm
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LinearRegression
from scipy.stats import ttest_ind
from scipy.stats import normaltest
% matplotlib notebook

//anaconda/lib/python3.5/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


### Load Data

In [3]:
all_data_store_path = '/Users/tylio/Google Drive/2018 spring/686/Wiley Sponsered Project/Data'
current_data_folder_name = '(40000+)_by_year'
current_data_store_path = os.path.join(all_data_store_path,current_data_folder_name)
data = pd.read_csv(os.path.join(current_data_store_path,'whole_file.csv'))

//anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (2,6,34,48,50,56,58,63) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Extract Keywords from Titles

In [123]:
import string
from collections import defaultdict, Counter
from itertools import chain, groupby, product

import nltk
from nltk.tokenize import wordpunct_tokenize
from nltk.stem import WordNetLemmatizer
class my_Rake(object):

    def __init__(self, stopwords=None, punctuations=None, language='english'):
        # If stopwords not provided we use language stopwords by default.
        self.stopwords = stopwords
        if self.stopwords is None:
            self.stopwords = nltk.corpus.stopwords.words(language)

        # If punctuations are not provided we ignore all punctuation symbols.
        self.punctuations = punctuations
        if self.punctuations is None:
            self.punctuations = string.punctuation

        # All things which act as sentence breaks during keyword extraction.
        self.to_ignore = set(chain(self.stopwords, self.punctuations))

        # Stuff to be extracted from the provided text.
        self.frequency_dist = None
        self.degree = None
        self.rank_list = None
        self.ranked_phrases = None

    def extract_keywords_from_text(self, text):
        sentences = nltk.tokenize.sent_tokenize(text)
        self.extract_keywords_from_sentences(sentences)

    def extract_keywords_from_sentences(self, sentences):
        phrase_list = self._generate_phrases(sentences)
        self._build_frequency_dist(phrase_list)
        self._build_word_co_occurance_graph(phrase_list)
        self._build_ranklist(phrase_list)

    def get_ranked_phrases(self):
        return self.ranked_phrases

    def get_ranked_phrases_with_scores(self):
        return self.rank_list

    def get_word_frequency_distribution(self):
        return self.frequency_dist

    def get_word_degrees(self):
        return self.degree

    def _build_frequency_dist(self, phrase_list):
        self.frequency_dist = Counter(chain.from_iterable(phrase_list))

    def _build_word_co_occurance_graph(self, phrase_list):
        co_occurance_graph = defaultdict(lambda: defaultdict(lambda: 0))
        for phrase in phrase_list:
            # For each phrase in the phrase list, count co-occurances of the
            # word with other words in the phrase.
            #
            # Note: Keep the co-occurances graph as is, to help facilitate its
            # use in other creative ways if required later.
            for (word, coword) in product(phrase, phrase):
                co_occurance_graph[word][coword] += 1
        self.degree = defaultdict(lambda: 0)
        for key in co_occurance_graph:
            self.degree[key] = sum(co_occurance_graph[key].values())

    def _build_ranklist(self, phrase_list):
        self.rank_list = []
        for phrase in phrase_list:
            rank = 0.0
            for word in phrase:
                rank += 1.0 * self.degree[word] / self.frequency_dist[word]
            self.rank_list.append((rank, ' '.join(phrase)))
        self.rank_list.sort(reverse=True)
        self.ranked_phrases = [ph[1] for ph in self.rank_list]

    def _generate_phrases(self, sentences):
        phrase_list = set()
        # Create contender phrases from sentences.
        lem = WordNetLemmatizer()
        for sentence in sentences:
            word_list = [lem.lemmatize(word.lower()) for word in word_tokenize(sentence)]
            phrase_list.update(self._get_phrase_list_from_words(word_list))
        return phrase_list

    def _get_phrase_list_from_words(self, word_list):
        groups = groupby(word_list, lambda x: x not in self.to_ignore and pos_tag([x])[0][1]!='VBG')
        return [tuple(group[1]) for group in groups if group[0]]

In [125]:
def keyword_extractor(data):
    extractor = my_Rake()
    extractor.extract_keywords_from_text(data['TI'])
    ti = ';'.join([i[1] for i in extractor.get_ranked_phrases_with_scores()])
    if type(data['DE'])==float:
        return ti
    else:
        extractor.extract_keywords_from_text(data['DE'])
        de = ';'.join([i[1] for i in extractor.get_ranked_phrases_with_scores()])
        return ti+';'+de

In [126]:
key_words = data.apply(lambda row: keyword_extractor(row),axis = 1)

In [116]:
data.loc[:,'key_words'] = key_words

### Filter out Important Keywords

In [127]:
kw_filter = set(chain(string.punctuation,stopwords.words('english')))

In [129]:
kw_candidate = ';'.join(list(data['key_words'])).split(';')
kw_pds_original = pd.Series(kw_candidate)

In [130]:
topics = list(kw_pds_original.value_counts()[kw_pds_original.value_counts()>100].index.values)

In [132]:
len(topics)

230

Finally, we got 230 candidate keywords

### Collect the Document Index for Each Topic

In [134]:
key_words_candidate = data['key_words'].apply(lambda x: ' '+(' ; ').join(x.split(';'))+' ')

In [135]:
data.loc[:,'key_words_candidate'] = key_words_candidate

In [136]:
topic_docIndex = {}
kwc = data['key_words_candidate']
for topic in topics:
    topic_docIndex[topic] = kwc[kwc.apply(lambda x: True if ' '+topic+' ' in x else False)].index.values

In [146]:
topic_map = pd.DataFrame(index = topic_docIndex.keys(),columns=['doc_index'])
for topic in topic_docIndex.keys():
    topic_map.loc[topic,'doc_index'] = topic_docIndex[topic]
topic_map = topic_map.reset_index().rename(columns = {'index':'topic'})

In [148]:
topic_map.head()

,topic,doc_index
0,polyester,"[105, 125, 274, 326, 392, 549, 560, 582, 652, ..."
1,investigation,"[302, 432, 656, 659, 777, 802, 827, 1038, 1148..."
2,biofilm,"[147, 159, 231, 272, 611, 678, 712, 837, 896, ..."
3,self-assembly,"[35, 163, 292, 460, 480, 523, 541, 545, 594, 5..."
4,medical device,"[23, 233, 256, 957, 1404, 2003, 2100, 2239, 22..."


### Eliminate the Topics with Similar Profiles

In [149]:
topic_doc_map_1gram = topic_map[topic_map['topic'].str.split(' ').apply(len)==1]
topic_doc_map_2gram = topic_map[topic_map['topic'].str.split(' ').apply(len)==2]
topic_doc_map_3gram = topic_map[topic_map['topic'].str.split(' ').apply(len)==3]

In [150]:
def similar_score(s1,s2):
    union1 = len(set(s1))
    union2 = len(set(s2))
    union = len(set(np.concatenate((s1,s2))))
    intersection = len(s1)+len(s2)-union
    score1 = intersection/union1
    score2 = intersection/union2
    return score1,score2

In [151]:
def duplication_detection(df1,df2):
    delete_dict = {}
    for index_1 in df1.index:
        for index_2 in df2.index:
            if df1['topic'][index_1] in df2['topic'][index_2]:
                s1 = df1['doc_index'][index_1]
                s2 = df2['doc_index'][index_2]
                score1,score2 = similar_score(s1,s2)
                threshold = 0.9
                if score1>threshold and score2>threshold:
                    if score1>=score2:
                        delete_dict[df1['topic'][index_1]] = True
                    else:
                        delete_dict[df2['topic'][index_2]] = True
#                 if score1>threshold:
#                     delete_dict[df1['topic'][index_1]] = True
#                 if score2>threshold:
#                     delete_dict[df2['topic'][index_2]] = True
    return delete_dict

In [152]:
def delete_duplication(original_df,delete_dict):
    transformed_df = original_df.copy()
    transformed_df = transformed_df[~transformed_df['topic'].isin(delete_dict.keys())]
    return transformed_df




def merge_duplications(original_df,similar_dict):
    transformed_df = original_df.copy()
    for old_topic in similar_dict:
        new_topic = similar_dict[old_topic]
        s1 = transformed_df.loc[transformed_df['topic']==old_topic,'doc_index'].values[0]
        s2 = transformed_df.loc[transformed_df['topic']==new_topic,'doc_index'].values[0]
        new_s = np.array(list(set(np.concatenate((s1,s2)))))
        index = transformed_df[transformed_df['topic']==new_topic].index.values[0]
        transformed_df.loc[index,'doc_index'] = new_s
    transformed_df = transformed_df[~transformed_df['topic'].isin(similar_dict.keys())]
    return transformed_df

In [153]:
topic_map = delete_duplication(topic_map,duplication_detection(topic_doc_map_2gram,topic_doc_map_3gram))
topic_map = delete_duplication(topic_map,duplication_detection(topic_doc_map_1gram,topic_doc_map_2gram))

In [30]:
#topic_map.to_csv('topic_map.csv',index = False)

In [155]:
for i in topic_map.index:
    data.loc[topic_map['doc_index'][i],topic_map['topic'][i]] = 1

In [62]:
#data.to_csv('wholeFile_with_wholeTopic_v3.csv',index = False)

### Get Features for Topic Map

In [156]:
# all_data_store_path = '/Users/tylio/Google Drive/2018 spring/686/Wiley Sponsered Project/Data'
# current_data_folder_name = '(40000+)_by_year'
# current_data_store_path = os.path.join(all_data_store_path,current_data_folder_name)
# data = pd.read_csv(os.path.join(current_data_store_path,'wholeFile_with_wholeTopic.csv'))
data = data[(data['PY']>=1995) & (data['PY']<2018)]
# topics = pd.read_csv(os.path.join(current_data_store_path,'topic_map.csv'))

In [157]:
def topic_percentage_yealy(df,topic):
    percentage = pd.concat([df.groupby('PY')[topic].count(),df.groupby('PY')['DA'].count()],axis = 1)
    percentage.loc[:,'percentage'] = percentage.apply(lambda x: x[topic]/x['DA'],axis = 1)
    return percentage['percentage']

def topics_trends_main(df,topic_map):
    i = 0
    for topic in topic_map['topic']:
        topic_percentage = topic_percentage_yealy(df,topic)
        for year in topic_percentage.index:
            topic_map.loc[topic_map['topic']==topic,str(int(year))] = topic_percentage[year]
        i+=1
        if i%10==0:
            print(i)
        

In [158]:
topics_trends_main(data,topic_map)

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220


In [159]:
for i in range(1995,2018):
    topic_map[str(i)+'(acc)'] = topic_map[str(i)]-topic_map['1995']

In [160]:
for i in topic_map.index:
    for j in range(1996,2018):
        sample = topic_map.loc[i,str(j)]-topic_map.loc[i,str(j-1)]
        topic_map.loc[i,str(j)+'(gap)'] = sample

In [161]:
topic_map['doc_index'] = topic_map['doc_index'].apply(lambda x: ';'.join([str(i) for i in x]))

In [164]:
topic_map.to_csv(os.path.join(current_data_store_path,'topic_map_v2_2.csv'),index = False)

In [165]:
data.to_csv(os.path.join(current_data_store_path,'wholeFile_with_wholeTopic_v2_2.csv'),index = False)